# CIFAR-10  HiCo Layer 1


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import cifar10
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K
import cv2

import os
import pickle
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
NUM_TN = 10
SUB_REGION_SCALE = 14
MIN_SUB_REGION_SCALE = 12
MAX_SUB_REGION_SCALE = 16
NUM_HICO_LAYER = 5

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [ ]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [ ]:
#Generate cropped train image
X_train_cropped_list = []
coordinate_list = []
scale_list = []
for i in range(NUM_TN):
  X_train_cropped = []
  sub_region_scale = random.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = random.randint(0, X_train.shape[1] - sub_region_scale)
  y = random.randint(0, X_train.shape[1] - sub_region_scale)
  coordinate = (x, y)
  for j in range(X_train.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_train[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_train_cropped.append(image_cropped)
  
  X_train_cropped = np.array(X_train_cropped)
  X_train_cropped_list.append(X_train_cropped)
  coordinate_list.append(coordinate)
  scale_list.append(sub_region_scale)

In [ ]:
#Generate cropped test image
X_test_cropped_list = []
for i in range(NUM_TN):
  X_test_cropped = []
  sub_region_scale = scale_list[i]
  x = coordinate_list[i][0]
  y = coordinate_list[i][1]
  for j in range(X_test.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_test[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_test_cropped.append(image_cropped)

  X_test_cropped = np.array(X_test_cropped)
  X_test_cropped_list.append(X_test_cropped)

In [ ]:
#rescaling dataset
for i in range(NUM_TN):
  temp = []
  for j in range (X_train.shape[0]):
    temp.append(cv2.resize(X_train_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_train_cropped_list[i] = temp

  temp = []
  for j in range(X_test.shape[0]):
    temp.append(cv2.resize(X_test_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_test_cropped_list[i] = temp

In [ ]:
#normalizing dataset
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i]/255
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [ ]:
#transform to vector
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [ ]:
#one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

## HiCo Layer 1

In [ ]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*X_train.shape[3]))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [ ]:
#train model
history = []
for i in range(NUM_TN):
  print('Model %d' %i)
  hist = ensemble[i][0].fit(X_train_cropped_list[i], y_train_one_hot, validation_data=(X_test_cropped_list[i], y_test_one_hot), epochs=5, batch_size=128)
  history.append(hist)

Model 0
Epoch 1/5
391/391 [==============================] - 2s 4ms/step - loss: 2.1386 - accuracy: 0.2055 - val_loss: 1.9915 - val_accuracy: 0.2801
Epoch 2/5
391/391 [==============================] - 1s 3ms/step - loss: 1.9340 - accuracy: 0.2955 - val_loss: 1.8851 - val_accuracy: 0.3160
Epoch 3/5
391/391 [==============================] - 1s 3ms/step - loss: 1.8740 - accuracy: 0.3172 - val_loss: 1.8300 - val_accuracy: 0.3414
Epoch 4/5
391/391 [==============================] - 1s 3ms/step - loss: 1.8351 - accuracy: 0.3318 - val_loss: 1.8074 - val_accuracy: 0.3442
Epoch 5/5
391/391 [==============================] - 1s 3ms/step - loss: 1.8133 - accuracy: 0.3441 - val_loss: 1.7921 - val_accuracy: 0.3520
Model 1
Epoch 1/5
391/391 [==============================] - 2s 3ms/step - loss: 2.1410 - accuracy: 0.2032 - val_loss: 1.9769 - val_accuracy: 0.2799
Epoch 2/5
391/391 [==============================] - 1s 3ms/step - loss: 1.9531 - accuracy: 0.2905 - val_loss: 1.8872 - val_accuracy: 0.31

In [ ]:
if not os.path.isdir('/content/drive/My Drive/fyp/') :
  os.mkdir('/content/drive/My Drive/fyp/')

with open('/content/drive/My Drive/fyp/CIFAR-10.pickle', 'wb') as f:
    pickle.dump([X_train_cropped_list, y_train_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list], f)

In [ ]:
for i in range(NUM_TN):
  ensemble[i][0].save_weights('/content/drive/My Drive/fyp/CIFAR-10_' + str(i) + '.h5')